In [32]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from tqdm import tqdm

# Step 1: train model

In [33]:
# load dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 28, 28).astype("float32") / 255
x_test = x_test.reshape(10000, 28, 28).astype("float32") / 255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")
#y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)
#y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)

# Reserve 10,000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [34]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28,1), name="in"))
model.add(tf.keras.layers.Dense(100, activation="relu", name="d1"))
model.add(tf.keras.layers.Dense(100, activation="relu", name="d2"))
model.add(tf.keras.layers.Dense(100, activation="relu", name="d3"))
model.add(tf.keras.layers.Dense(100, activation="relu", name="d4"))
model.add(tf.keras.layers.Dense(100, activation="relu", name="d5"))
model.add(tf.keras.layers.Dense(10, name="out"))
#model.summary()

In [35]:
def softmax_loss():        
    def loss(y_true, y_pred):
        y_soft = tf.keras.activations.softmax(y_pred, axis=1)
        loss_soft=keras.losses.SparseCategoricalCrossentropy()(y_true, y_soft)
        return loss_soft
    return loss

In [36]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=softmax_loss(), 
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [40]:
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=30,
    validation_data=(x_val, y_val),
    verbose=0,    
)

In [41]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=1)

313/313 [==============================] - 0s 1ms/step - loss: 0.1395 - sparse_categorical_accuracy: 0.9765


In [43]:
print(test_acc)

0.9764999747276306


In [44]:
model.save('MNIST_5x100.h5')

# Step 2: calculate jacobian

In [45]:
# calculate jacobian -- single input 
def get_jac(inp):
    actis = {}
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(inp)
        a = inp
        for layer in model.layers:
            curr_a = layer(a)
            actis[layer.name] = curr_a
            a = curr_a
        y = a

    jacobian = {}
    for key in actis:
        a = actis[key]
        dy_da = tape.jacobian(y, a)
        new_shape = [dy_da.shape[i] for i in range(0, len(dy_da.shape)) if dy_da.shape[i] != 1]
        dy_da = tf.reshape(dy_da, shape=tuple(new_shape))
        jacobian[key] = dy_da.numpy()
    return jacobian

# calculate jacobian -- multiple inputs
def get_jacobian(inputs):
    all_jacs = []
    for i in tqdm(range(inputs.shape[0])):
        inp = inputs[i]
        inp = tf.expand_dims(inp, axis=0)
        one_jac = get_jac(inp)
        all_jacs.append(one_jac)
    
    jacobian = {}
    for k in all_jacs[0].keys():
        jacobian[k] = sum(jac[k] for jac in all_jacs)
    return jacobian

In [46]:
num_samples=30
all_jacobians = {}
for i in range(10):
    x_test_labeli = tf.squeeze(x_test[tf.where(y_test == float(i))])
    inpt = x_test_labeli[0:num_samples]
    jacobian = get_jacobian(inpt)
    all_jacobians[i] = jacobian

  0%|                                                                                                                                                                                        | 0/30 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:29<00:00,  1.02it/s]


In [47]:
import pickle
with open('MNIST_5x100_jacobian.pkl', 'wb') as f:
    pickle.dump(all_jacobians, f)

# Step 3: combine Jacobians of different labels

In [33]:
import pickle
with open('MNIST_5x100_jacobian.pkl', 'rb') as f:
    all_jacobians = pickle.load(f)

In [34]:
num_classes = 10
denoised_jacobian = {}

# shape = (#layers, #neurons_p_layer, #out_neurons)
sample = list(all_jacobians.values())[0]
for layer in sample.keys():
    denoised_jacobian[layer] = np.zeros(sample[layer].shape)
    for idx in all_jacobians.keys():
        curr_jac = all_jacobians[idx][layer]
        denoised_jacobian[layer][idx] = curr_jac[idx]

In [35]:
print(denoised_jacobian['d4'])

[[ 7.77160108e-01  8.47812843e+00  1.46541846e+00  1.68446312e+01
   1.06298819e+01 -2.76039815e+00  8.65498829e+00 -5.96987820e+00
   1.41170301e+01 -3.25745082e+00 -3.35684514e+00  7.66733551e+00
   2.37549841e-01  1.02647352e+01  6.37810409e-01  4.03422266e-01
   1.05678558e+01  1.80823457e+00 -6.29241276e+00  2.32379174e+00
  -2.21960163e+00  1.85847354e+00 -4.97266144e-01 -1.19671326e+01
   7.51419163e+00  7.67845201e+00 -1.80693853e+00  1.25767493e+00
  -3.54597425e+00 -3.43354559e+00  1.16957750e+01 -2.36712980e+00
  -4.09087032e-01 -4.52231836e+00  1.22435665e+01 -3.11471486e+00
  -1.79171562e+00  1.40001373e+01 -7.24811363e+00 -4.60525131e+00
   1.67260380e+01  2.65542960e+00 -1.16848493e+00  1.16995382e+01
  -2.07567620e+00 -1.89747345e+00  1.17800701e+00 -6.93317270e+00
   3.48224258e+00 -3.57830954e+00 -8.03823948e+00  4.70192957e+00
  -1.26103010e+01 -3.42622066e+00  3.56041074e+00  1.29658661e+01
   5.53646183e+00 -2.57206154e+00  3.61554384e+00  3.97123456e+00
   2.16030

# Step 4: make jacobian into distance matrix

In [36]:
tf.executing_eagerly()

True

In [37]:
from sklearn.metrics.pairwise import cosine_distances
def pairwise_cosine_dist(A):
    D = cosine_distances(A)
    return D

In [38]:
def get_distances(jacobian):
    distances = {}
    for k in jacobian.keys():
        if k != 'in' and k != 'out':
            j = jacobian[k].T
            distances[k] = pairwise_cosine_dist(j)
    return distances

In [ ]:
IDX = 2
curr_jac = all_jacobians[IDX] #denoised_jacobian
for key in curr_jac.keys():
    curr_jac[key] -= np.amin(curr_jac[key])
    curr_jac[key] /= np.amax(curr_jac[key])
distances = get_distances(curr_jac)
print([(key, distances[key].shape) for key in distances.keys()])

# Step 4: cluster net

In [124]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #for no tensorflow warnings"
sys.path.append(os.path.abspath('..'))

In [125]:
from src import *
from models import *
from src.main import Cluster_Class, set_file_model, set_keras_model
from src.models import Keras_Model
from src.cluster_methods import SensitivityAnalysis

In [126]:
def softmax_loss():        
    def loss(y_true, y_pred):
        y_soft = tf.keras.activations.softmax(y_pred, axis=1)
        loss_soft=keras.losses.SparseCategoricalCrossentropy()(y_true, y_soft)
        return loss_soft
    return loss

In [136]:
model = tf.keras.models.load_model('MNIST_5x100.h5', custom_objects={'loss': softmax_loss()})

In [137]:
m = set_keras_model(model)
m.update_keras()
print("model accuracy. ", m.test_accuracy(verbose=False))
pre_acc = []
for i in range(0, 10):
    pre_acc.append(m.test_MNIST_labelacc(i))
print("pre_acc: ", m.test_MNIST_labelacc(IDX))

model accuracy.  0.9764999747276306
pre_acc:  0.9670542478561401


In [138]:
eps = {1: 1e-7, 2:1e-7, 3:1e-7, 4:0.01, 5:0.023}
SA = SensitivityAnalysis(distances = distances, eps=eps)

In [139]:
cc = Cluster_Class(m, [10], cl_method="gradients", test_method=SA)
acc, dic = cc.perform_clustering(verbose=True)
print(acc, dic)
print("model accuracy on class {0}. ".format(IDX), m.test_MNIST_labelacc(IDX))

Test set accuracy:  0.9764999747276306
----- [Get Activations] -----
----- [Start Clustering] -----
   - Layer 1
----- [Get Clusters] -----
[]
   - Layer 2
----- [Get Clusters] -----
[]
   - Layer 3
----- [Get Clusters] -----
[]
   - Layer 4
----- [Get Clusters] -----
[(0, array([ 1, 82]), 1, 0, 38.898407), (1, array([17, 62]), 17, 0, 26.370289), (2, array([18, 68]), 18, 0, 32.253258), (3, array([42, 84]), 42, 0, 50.242622), (4, array([73, 98]), 73, 0, 37.72207)]
----- [Apply the Clustering to the Network] -----
   - Layer 5
----- [Get Clusters] -----
[(0, array([ 0,  4, 16, 22, 24, 27, 29, 30, 36, 39, 40, 55, 56, 61, 68, 69, 75,
       79, 92, 93, 96, 99]), 56, 0, 133.48192), (1, array([ 1, 13, 18, 44, 62, 76]), 44, 0, 143.2701), (2, array([ 5, 10]), 5, 0, 105.34001), (3, array([ 9, 26, 32, 42, 46, 88]), 26, 0, 138.94534), (4, array([19, 21, 67, 95]), 95, 0, 149.37738), (5, array([33, 43, 86]), 43, 0, 112.867775), (6, array([51, 84]), 51, 0, 69.76188), (7, array([52, 78, 82]), 78, 0, 

In [140]:
post_acc = []
for i in range(0, 10):
    post_acc.append(m.test_MNIST_labelacc(i))

for i in range(0, 10):
    print(i, " -> ", pre_acc[i]-post_acc[i])

0  ->  0.012244880199432373
1  ->  0.0008810758590698242
2  ->  0.02325582504272461
3  ->  -0.012871325016021729
4  ->  0.0641547441482544
5  ->  0.020179331302642822
6  ->  -0.001043856143951416
7  ->  0.0
8  ->  0.0
9  ->  -0.004955410957336426
